<a href="https://colab.research.google.com/github/lhubbard88/ml-class/blob/master/Copy_of_bagging_n_pasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bagging and Pasting

#### Part of the [Inquiryum Machine Learning Fundamentals Course](http://inquiryum.com/machine-learning/)

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/bagging..png)


Now we are about to embark on our journey from simple decision trees to algorithms that use decision trees as components. The path goes like this:


![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/dbxg.png)

The use of decision trees began in the 1980s and XGBoost was introduced in 2016. Throughout the next few notebooks we will explore this progression of algorithms.  

### A collective of classifiers

To gain an intuition on how this works, let's look at how our confidence might increase when more people tell us something. Whether it is multiple doctors giving us the same diagnosis or something as simple as ...

#### The Mary Spender example

Let's say one of your friends mentions over lunch (pre-covid apparently) that you would love a particular musical artist on YouTube, say Mary Spender, who you never heard before. 

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/MarySpender2.png)

What is the chance that you will actually like Mary Spender's music? Maybe slighly better than chance? Let's say you think there is a 60% chance you will like her. You will file away the recommendation but you are not going to rush home and watch a YouTube video.  Now, in addition to the lunch friend's recommendation,  an old music school friend, now living in Austin messages you saying you should check out Mary Spender and the friend predicts you will absolutely love her. Then a week later, while talking with an old bandmate over the phone, that bandmate, again, recommends Mary Spender. Over the course of less than 10 days, three of your friends independently (because they don't know one another) recommend Mary Spender. Now what is the likelihood of you liking Mary Spender? I am guessing you think that now it is higher than 60%. Maybe now you think it is 90% likely you will like her. It is the aggregate of these 3 people's opinions (3 classifiers) that ups the accuracy of the prediction.

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/spender22.png)


This is similar to how bagging works. One aggregates the votes of a number of classifiers and the vote of that ensemble of classifiers is more accurate than that of a single classifier. Even if the accuracy of each component classifier is low (known as a weak classifier), the ensemble can be a strong (high accuracy) classifier. Of course there are some caveats. 

Back to the Mary Spender example. Suppose one of your friends went to a Mary Spender concert and then later in the week met with four other of your friends and mentioned that she thought you would love Mary Spender's music. Then, over the course of a week all those friends recommended Mary Spender to you. In this case the recommendations are not that independent---all are based on one person's opinion. Thus, the accuracy would not be as great as in the example above. Similarly, if you made 10 copies of the exact same classifier each trained on exactly the same data, the accuracy of the ensemble of clones would not be any better than the accuracy of a single copy. Moving away from Mary Spender and our musical tastes and back to machine learning, we can try to create independence among the classifier in 2 ways:

1. We can change the type of classifier. For example, we can use a k-Nearest Neighbor Classifier with Manhattan distance and a k of 5, a k-Nearest Neighbor Classifier with Euclidean distance and a k of 3, a decision tree classifier using entropy and a max depth of 5, and a decision tree classifier with using gini and no max depth specified. Hopefully, the accuracy of the ensemble of the four classifiers would be greater than that of a single classifier.
2. We can have an ensemble of the same classifier (for example, 10 decision tree classifiers with identical hyperparameters) but each classifier can get a different subset of the training data. The classifiers would thus build different models (differents 'rules') and, again, the accuracy of the ensemble should be greather than that of a single classifier. This is the approach we will take.

### Bagging and Pasting


In this Jupyter notebook, we are going to explore Bagging algorithms. Bagging algorithms come in a variety of 'flavors' including one called 'bagging' and one called 'pasting'.

But first an experiment on the what *with replacement* means. As you will see shortly, that term is the crucial difference between bagging and pasting.

### A small experiment
NOTE: The following code is just used for illustration and is nothing we will be using for machine learning. 

Consider a list of 5 red balls and 5 blue balls:

In [ ]:
bag = ['red', 'red', 'red', 'red', 'red',
       'blue', 'blue', 'blue', 'blue', 'blue']

Suppose we want to pick 7 random balls from this list. Python offers two functions that will give us random elements from a list.One is called `choices` which selects a sample with replacement, which means that once a ball is selected it is put back in the bag so it has the potential to be selected again. Let's give it a try, and just because things are random let's do this 100 times:

In [ ]:
import random
total = 0
for i in range(100):
    set = random.choices(bag, k=7)
    blue = set.count('blue')
    red = set.count('red')
    if blue > 5 or red > 5:
        print("%i blue and %i red" % (set.count('blue'), set.count('red')))
        total +=1
print("Balls selected exceeded balls in bag: %i" % (total))

6 blue and 1 red
6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
1 blue and 6 red
1 blue and 6 red
1 blue and 6 red
1 blue and 6 red
Balls selected exceeded balls in bag: 8


*A reminder: please don't mindlessly execute the code. Look at it and understand it*

When I ran this, 14 times out of 100 had more of one color ball than there were in the original bag. In fact, several times I ended up with all 7 of the balls blue, even though the original list had only 5 balls:

```
7 blue and 0 red
6 blue and 1 red
7 blue and 0 red
1 blue and 6 red
0 blue and 7 red
1 blue and 6 red
1 blue and 6 red
6 blue and 1 red
6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
6 blue and 1 red
Balls selected exceeded balls in bag: 14
```
Again, this is called selecting with replacement (we put what we selected back in the set before selecting again). 

The other alternative is to select without replacement--once we select something we can't select it again. Python's `sample` does this:


In [ ]:
import random
total = 0
for i in range(1000):
    set = random.sample(bag, k=7)
    blue = set.count('blue')
    red = set.count('red')
    if blue > 5 or red > 5:
        print("%i blue and %i red" % (set.count('blue'), set.count('red')))
        total +=1
print("Balls selected exceeded balls in bag: %i" % (total))

Balls selected exceeded balls in bag: 0


As you can see, the number of a specific colored ball that we select never exceeded the number of balls of that color in the original set.

Now back to bagging and pasting. In both approaches we are going to sample the training data. Let's say we want 70% of the training data in our sample. In bagging ([Breiman, 1996](https://link.springer.com/content/pdf/10.1007/BF00058655.pdf)), if we our training dataset is 1000 instances and we want 70% for a particular classifier, the algorithm will randomly select 700 out of the 1,000 **with replacement**. With pasting ([Breiman, 1998](https://link.springer.com/article/10.1023/A:1007563306331)), the selection is done **without replacement**. 

#### but wait, there is more ...

There are two other options. Instead of selecting a random subset of training data instances, we can select a random subset of columns (features). Let's say we have a dataset of 1,000 instances each with 100 features. When we select a random subset of columns, we still have 1,000 instances but now they have just a subset of the features. This is called Random Subspaces ([Ho, 1998](https://pdfs.semanticscholar.org/b41d/0fa5fdaadd47fc882d3db04277d03fb21832.pdf?_ga=2.196949164.1638238666.1596910000-1073138517.1596910000)).

Finally, we can train a classifier on both random subsets of instances and random subsets of features. This is known as Random Patches ([Louppe and Geurts, 2012](https://www.researchgate.net/publication/262212941_Ensembles_on_Random_Patches))

In summary, the four methods are:

* **bagging** - select a subset of data set instances using replacement
* **pasting** - select a subset of data set instances without replacement
* **Random Subspaces** - select a subset of features
* **Random Patches** - select both a subset of features and of instances

Let's see how this works!

First, let's grab the Wisconsin Cancer data we used before:

#### Wisconsin Cancer Dataset
![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/aimam.png)
(image from Nvidia's [AI Improves Breast Cancer Diagnoses by Factoring Out False Positives](https://blogs.nvidia.com/blog/2018/02/01/making-mammography-more-meaningful/))

[A description of the Cancer Database](#Breast-Cancer-Database)

In this dataset we are trying to predict the diagnosis---either M (malignant) or B (benign).

Let's load the dataset and split it into training and testing sets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

colNames = ['id', 'diagnosis', 'radiusAvg', 'textureAvg', 'perimeterAvg', 'areaAvg',
            'smoothnessAvg', 'compactnessAvg', 'concavityAvg', 'concavityPointsAvg',
            'symmetryAvg', 'FractalDimensionAvg', 'radiusSE', 'textureSE', 'perimeterSE',
            'areaSE','smoothnessSE', 'compactnessSE', 'concavitySE', 'concavityPointsSE',
            'symmetrySE', 'FractalDimensionSE', 'radiusWorst', 'textureWorst', 'perimeterWorst',
            'areaWorst', 'smoothnessWorst', 'compactnessWorst', 'concavityWorst', 'concavityPointsWorst',
            'symmetryWorst>', 'FractalDimensionWorst']
len(colNames)

data = pd.read_csv('https://raw.githubusercontent.com/zacharski/ml-class/master/data/wdbc.data', names=colNames)
data.set_index('id', inplace=True)

trainingdata, testdata = train_test_split(data, test_size = 0.2)
testdata

,diagnosis,radiusAvg,textureAvg,perimeterAvg,areaAvg,smoothnessAvg,compactnessAvg,concavityAvg,concavityPointsAvg,symmetryAvg,FractalDimensionAvg,radiusSE,textureSE,perimeterSE,areaSE,smoothnessSE,compactnessSE,concavitySE,concavityPointsSE,symmetrySE,FractalDimensionSE,radiusWorst,textureWorst,perimeterWorst,areaWorst,smoothnessWorst,compactnessWorst,concavityWorst,concavityPointsWorst,symmetryWorst>,FractalDimensionWorst
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
871122,B,12.06,12.74,76.84,448.6,0.09311,0.05241,0.01972,0.019630,0.1590,0.05907,0.1822,0.7285,1.171,13.25,0.005528,0.009789,0.008342,0.006273,0.01465,0.002530,13.14,18.41,84.08,532.8,0.12750,0.12320,0.08636,0.07025,0.2514,0.07898
869931,B,13.74,17.91,88.12,585.0,0.07944,0.06376,0.02881,0.013290,0.1473,0.05580,0.2500,0.7574,1.573,21.47,0.002838,0.015920,0.017800,0.005828,0.01329,0.001976,15.34,22.46,97.19,725.9,0.09711,0.18240,0.15640,0.06019,0.2350,0.07014
922577,B,10.32,16.35,65.31,324.9,0.09434,0.04994,0.01012,0.005495,0.1885,0.06201,0.2104,0.9670,1.356,12.97,0.007086,0.007247,0.010120,0.005495,0.01560,0.002606,11.25,21.77,71.12,384.9,0.12850,0.08842,0.04384,0.02381,0.2681,0.07399
84667401,M,13.73,22.61,93.60,578.3,0.11310,0.22930,0.21280,0.080250,0.2069,0.07682,0.2121,1.1690,2.061,19.21,0.006429,0.059360,0.055010,0.016280,0.01961,0.008093,15.03,32.01,108.80,697.7,0.16510,0.77250,0.69430,0.22080,0.3596,0.14310
921386,B,14.47,24.99,95.81,656.4,0.08837,0.12300,0.10090,0.038900,0.1872,0.06341,0.2542,1.0790,2.615,23.11,0.007138,0.046530,0.038290,0.011620,0.02068,0.006111,16.22,31.73,113.50,808.9,0.13400,0.42020,0.40400,0.12050,0.3187,0.10230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.053020,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.037310,0.047300,0.015570,0.01318,0.003892,18.98,34.12,126.70,1124.0,0.11390,0.30940,0.34030,0.14180,0.2218,0.07820
91903902,B,13.68,16.33,87.76,575.5,0.09277,0.07255,0.01752,0.018800,0.1631,0.06155,0.2047,0.4801,1.373,17.25,0.003828,0.007228,0.007078,0.005077,0.01054,0.001697,15.85,20.20,101.60,773.4,0.12640,0.15640,0.12060,0.08704,0.2806,0.07782
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.127900,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.040060,0.038320,0.020580,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.14440,0.42450,0.45040,0.24300,0.3613,0.08758


Now divide up the data into the features and labels

In [ ]:
colNames.remove('id')
colNames.remove('diagnosis')
trainingDataFeatures = trainingdata[colNames]
testDataFeatures = testdata[colNames]
trainingDataLabels = trainingdata['diagnosis']
testDataLabels = testdata['diagnosis']


Let's get a base accuracy using a single decision tree classifier:

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = clf.predict(testDataFeatures)

accuracy_score(testDataLabels, predictions)

0.9824561403508771

### Building a bagging classifier

Let's build a collective of 20 decision tree classifiers (`n_estimators`). Let's train each one with 100 random samples from our dataset (`max_samples`) with replacement (`bootstrap=True`). `n_jobs` means how many jobs to run in parallel. `n_jobs=-1` means use all available CPU cores.   

Just to reinforce the vocabulary we are learning, `n_estimators`, `max_samples`, `bootstrap` are among the **hyperparameters** of the bagging classifier.

In [ ]:
from sklearn.ensemble import BaggingClassifier
clf = tree.DecisionTreeClassifier(criterion='entropy')

bagging_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
bagging_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = bagging_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9385964912280702

When I did this using a single decision tree classifier was 90.3% accurate, while the bagging classifier was 96.5% accurate--halving the error rate! that's pretty good!


### Pasting
Let's try the same thing with pasting (without replacement):

For that we set the hyperparameter: `bootstrap=False`


In [ ]:
pasting_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=False, n_jobs=-1)
pasting_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = pasting_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9649122807017544

### Random Subspaces
Again, random subspaces are when we randomly select feature subsets rather than subsets of the dataset instances. This time we will create 50 classifiers for our collective and each will train on a dataset with 7 features (`max_feature=7`).

In [ ]:
subspace_clf = BaggingClassifier(clf, n_estimators=50, max_features=7, 
                                bootstrap=True, n_jobs=-1)
subspace_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = subspace_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9912280701754386

### Random Patches
Finally, let's combine things and try random patches. In this example each classifier will be given a subset of 100 training instances with 7 features each.:

In [ ]:
subspace_clf = BaggingClassifier(clf, n_estimators=100, max_features=7, 
                                 max_samples=100, bootstrap=False, n_jobs=-1)
subspace_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = subspace_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9736842105263158

While it is common to use a decision tree as the base classifier, we can use any classifier. Here we use kNN:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
kNN = KNeighborsClassifier()
bagging_clf = BaggingClassifier(kNN, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
bagging_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = bagging_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9385964912280702

#### Summary
As you can see, any of these simple bagging algorithms typically outperforms using a single classifier. 


### Review

We import the bagging classifier library with:

```
from sklearn.ensemble import BaggingClassifier
```

and create an instance of one with:

```
my_bagging_classifier = BaggingClassifier(baseClassifier, Hyperparameters,n_jobs=-1)
```

#### Base Classifier
while any classifier can be used we typically use a decision tree

#### Hyperparameters
Here is a list of the hyperparameters (from the [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier)):

* `n_estimators`: integer, default value = 10, the number of classifiers (estimators) in the ensemble.
* `max_samples`: integer or float, default value = 1.0(meaning use all the training instances), the number of samples (instances) to draw from the training dataset to train each base classifier.
    * if integer, then draw max_features features.
    * if float, then draw max_samples * X.shape[0] samples. For example if `max_samples` is 0.7 and there are 100 instances in the training dataset then draw 70 samples.
* `max_features`, integer or float, default value =1.0,  
the number of features to draw from the training dataset to train each base estimator 
    * if integers, then draw max_features features.
    * if float, then draw max_features * X.shape[1] features.
* `bootstrap` boolean, default value =True, whether samples and features are drawn with replacement. If False, sampling without replacement is performed.

For other hyperparamters, consult the documentation.

## You try: Predicting musical genres from audio file attributes

When you listen even to a few seconds of a song you can identify it as blues, country, classical, or any other genre. How do you do this? What attributes are you hearing in the audio file that helps you make this classification? And, more to the point, can we train a computer to do it?

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/bluesClassical.png)

We are going to be using  the [GTZAN Dataset for Music Genre Classification](https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification). It provides data of 100 songs for each of 10 genres. The data is in several formats:

* 30 second audio files (wav)
* spectral images of those 30 second clips (see image above)
* a csv file containing acoustic attributes of the 30 second clip
* a csv file containing acoustic attributes of 3 second clips (the 30 second clips were split into 3 second ones)

We are going to use the 3 second csv file which is available at 

https://raw.githubusercontent.com/zacharski/ml-class/master/data/gtzan.csv

Go ahead and load the data into a dataframe (the first row contains feature names)


In [ ]:
music = pd.read_csv("https://raw.githubusercontent.com/zacharski/ml-class/master/data/gtzan.csv")
music

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,perceptr_mean,perceptr_var,tempo,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,mfcc5_mean,mfcc5_var,mfcc6_mean,mfcc6_var,mfcc7_mean,mfcc7_var,mfcc8_mean,mfcc8_var,mfcc9_mean,mfcc9_var,mfcc10_mean,mfcc10_var,mfcc11_mean,mfcc11_var,mfcc12_mean,mfcc12_var,mfcc13_mean,mfcc13_var,mfcc14_mean,mfcc14_var,mfcc15_mean,mfcc15_var,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,1.080790e+06,0.081851,0.000558,-0.000078,0.008354,-0.000068,0.005535,129.199219,-118.627914,2440.286621,125.083626,260.956909,-23.443724,364.081726,41.321484,181.694855,-5.976108,152.963135,20.115141,75.652298,-16.045410,40.227104,17.855198,84.320282,-14.633434,83.437233,10.270527,97.001335,-9.708279,66.669891,10.183875,45.103611,-4.681614,34.169498,8.417439,48.269444,-7.233477,42.770947,-2.853603,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,6.722448e+05,0.087173,0.001030,-0.000099,0.004950,-0.000103,0.004854,123.046875,-125.590706,2038.344238,122.421227,216.774185,-20.718019,231.979767,50.128387,142.700409,-11.333302,139.243118,21.385401,77.817947,-15.960796,97.364029,19.454103,57.948093,-12.465918,68.271523,17.898169,56.222176,-11.732554,54.373909,8.145000,40.662876,-7.717751,30.808521,8.397150,48.784225,-8.300493,68.584824,4.074709,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,7.907127e+05,0.071383,0.000425,-0.000066,0.012476,0.000065,0.004357,123.046875,-132.441940,3798.532227,115.085175,257.321289,-14.811666,192.448074,50.189293,144.166031,-0.680819,128.376892,24.650375,66.371170,-13.506104,89.319336,15.643386,55.253967,-13.216637,120.308784,10.406025,35.757862,-7.991465,47.911613,11.853963,36.569931,-4.677677,40.725075,6.571110,30.686846,-2.424750,50.313499,4.806280,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,9.216524e+05,0.069426,0.000304,-0.000014,0.008318,0.000018,0.005927,123.046875,-118.231087,2508.781006,132.116501,332.650574,-18.758335,109.357529,39.769306,184.693344,-13.260426,144.398224,20.468134,122.516464,-14.563448,68.937332,18.745104,74.748886,-13.755463,73.868576,12.993759,41.549564,-12.648887,58.540478,10.389314,39.102024,-4.362739,60.714748,9.156193,40.411537,-9.889441,44.666325,-1.359111,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,6.102111e+05,0.070095,0.000289,0.000041,0.009634,-0.000106,0.005833,123.046875,-105.968376,2118.919922,134.643646,219.562622,-19.961748,171.878754,40.171753,103.120712,-14.271939,102.651230,18.734617,79.070000,-15.619381,48.510284,19.207966,53.642956,-18.274683,95.300995,14.316693,58.821163,-5.792194,55.030254,17.045437,43.229939,-5.681399,46.515259,5.705521,24.956211,-7.986080,39.816933,2.092937,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

Let's examine the values of the label column (the genres):

In [ ]:
music.label.unique()

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

Those are the 10 genres we are trying to predict. So if we were just to guess without hearing the clip, we would be accurate 10% of the time. How accurate do you think you would be based on hearing a 3 second clip? I am pretty confident I could correctly label the 30 second clips, but I am much less confident about labeling 3 second ones. Since guessing randomly would give me 10% accuracy, I am estimating maybe 50-60% accuracy. Let's see how a computer does.

#### Feature Names
So the column we are trying to predict is `label`. Now let's get the names of the feature columns

In [ ]:
featureNames = list(music.columns)
featureNames.remove('filename')
featureNames.remove('label')
print(featureNames)
print(len(featureNames))

['length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var', 'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean', 'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var', 'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo', 'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean', 'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var', 'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean', 'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean', 'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var']
58


So we have 58 features. 

#### Training and test sets
Now it is time to construct the training and test sets:

In [ ]:
## divide the original data 80% going into the music_training dataset 
## the rest in music_test
music_training, music_test = train_test_split(music, test_size = 0.2)
                              
## now create the DataFrames for just the features (excluding the label column 
## filename column)        
cols = list(music.columns.values.tolist())
cols.remove('filename')
cols.remove('label')                      
music_training_features = music_training[cols]
music_test_features = music_test[cols]
                              
## now create the labels data structure for both the training and test sets                              
music_training_labels = music_training['label']
music_test_labels = music_test['label']
music_test_labels

3126        disco
7422          pop
4568       hiphop
1993    classical
9331         rock
          ...    
4030       hiphop
426         blues
8382       reggae
3107        disco
4454       hiphop
Name: label, Length: 1998, dtype: object

### Building a single decision tree classifier
Let's build a single decision tree classifier called `clf` using entropy, fit it to the data, make predictions and determine the accuracy:

In [ ]:
## Create clf, an instance of the Decision Tree Classifier
clf = tree.DecisionTreeClassifier(criterion='entropy')

## Fit it to the data
clf.fit(music_training_features, music_training_labels)

## get the predictions for the test set
predictions = clf.predict(music_test_features)

## get the accuracy score
accuracy_score(music_test_labels, predictions)

0.6731731731731732

When I did this I got 66% accuracy. That doesn't sound great but keep in mind that random guessing would only be 10% accuracy. 

### Building a Random Patch Classifier

Now we are going to build a random patch classifier.

* the base classifier will be a decision tree using entropy
* the ensemble will contain 20 base classifiers
* each classifier will use a random sample of 70% of the training data
* each classifier will use a random sample of 70% of the features
* the sampling will be done with replacement
* it will use all available cpu cores.

We are going to

1. build the classifier
2. train the classifier on the data
3. make predictions on the test set
4. determine the accuracy

In [ ]:
subspace_clf = BaggingClassifier(clf, n_estimators=20, max_features=0.7, 
                                 max_samples=0.7, bootstrap=True, n_jobs=-1)
subspace_clf.fit(music_training_features, music_training_labels)
predictions = subspace_clf.predict(music_test_features)
accuracy_score(music_test_labels, predictions)

0.8373373373373373

What accuracy did you get? Was it better than using a single classifier?
Keep your original code above. Make a copy of it below and 
experiment a bit with the hyperparameters. (try 3 or 4 different things) What is the best accuracy you can get?

In [ ]:
subspace_clf = BaggingClassifier(clf, n_estimators=100, max_features=0.7, 
                                 max_samples=0.7, bootstrap=True, n_jobs=-1)
subspace_clf.fit(music_training_features, music_training_labels)
predictions = subspace_clf.predict(music_test_features)
accuracy_score(music_test_labels, predictions)

0.8783783783783784

In [ ]:
subspace_clf = BaggingClassifier(clf, n_estimators=100, max_features=0.7, 
                                 max_samples=0.7, bootstrap=False, n_jobs=-1)
subspace_clf.fit(music_training_features, music_training_labels)
predictions = subspace_clf.predict(music_test_features)
accuracy_score(music_test_labels, predictions)

0.8843843843843844

In [ ]:
subspace_clf = BaggingClassifier(clf, n_estimators=100, max_features=15, 
                                 max_samples=2000, bootstrap=False, n_jobs=-1)
subspace_clf.fit(music_training_features, music_training_labels)
predictions = subspace_clf.predict(music_test_features)
accuracy_score(music_test_labels, predictions)

0.8183183183183184

## NOTES

### Breast Cancer Database

[back](#Wisconsin-Cancer-Dataset)

  This breast cancer databases was obtained from the University of Wisconsin
   Hospitals, Madison from Dr. William H. Wolberg.  If you publish results
   when using this database, then please include this information in your
   acknowledgements.  Also, please cite one or more of:

   1. O. L. Mangasarian and W. H. Wolberg: "Cancer diagnosis via linear 
      programming", SIAM News, Volume 23, Number 5, September 1990, pp 1 & 18.

   2. William H. Wolberg and O.L. Mangasarian: "Multisurface method of 
      pattern separation for medical diagnosis applied to breast cytology", 
      Proceedings of the National Academy of Sciences, U.S.A., Volume 87, 
      December 1990, pp 9193-9196.

   3. O. L. Mangasarian, R. Setiono, and W.H. Wolberg: "Pattern recognition 
      via linear programming: Theory and application to medical diagnosis", 
      in: "Large-scale numerical optimization", Thomas F. Coleman and Yuying
      Li, editors, SIAM Publications, Philadelphia 1990, pp 22-30.

   4. K. P. Bennett & O. L. Mangasarian: "Robust linear programming 
      discrimination of two linearly inseparable sets", Optimization Methods
      and Software 1, 1992, 23-34 (Gordon & Breach Science Publishers).

Title: Wisconsin Breast Cancer Database (January 8, 1991)


Sources:
   -- Dr. WIlliam H. Wolberg (physician)
      University of Wisconsin Hospitals
      Madison, Wisconsin
      USA
   -- Donor: Olvi Mangasarian (mangasarian@cs.wisc.edu)
      Received by David W. Aha (aha@cs.jhu.edu)
   -- Date: 15 July 1992